 <img align="Center" width="900" height="20" src="san-franscico-houses.jpg">


# House Prices Prediction

Predecir el precio de una casa no es una tarea sencilla teniendo en cuenta la infinidad de característica que se podrían tener en cuenta, el material de construcción, el número de habitaciones, la cantidad de baños, la zona, el año de construcción, etc. Gracias a las técnias de aprendizaje automático exiten modelos que nos permiten realizar predicciones, pero antes se debe realizar un estudio exaustivo de la información disponible, una seleccion de variables, identificar si existe multicolinealidad, si es necesario normalizar algunas variables, etc. Luego de realizar el anális exploratorio de los datos se puede proceder a ajustar el modelo para luego evaluar su performace.

Para poder aplicar estos modelos es necesario contar con un conjunto de datos con el que se pueda explicar el problema. A priori no conocemos cuáles son la características más adecuadas para el modelo por lo tanto es necesario realizar análisis exploratorio.

###  Exploración de los datos

El primer paso que se realiza es la importación de los datos y la exploración de las variables disponibles. El tipo de variables, si es necesario realizar alguna conversión y la cantidad de valores faltantes.

In [61]:
#Se importan las liberías
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

In [86]:
# Se cargan los datos
house_price=pd.read_csv('train.csv')
house_price=house_price.drop(columns="Id")
house_price.shape

(1460, 80)

Una vez importada la base de datos se separa la variable de respuesta **SalePrice** del resto de la base de datos y se realiza una primera exploración.

In [125]:
y_hp=house_price['SalePrice']
x_hp=house_price.drop(columns='SalePrice')

y_hp.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [63]:
#x_hp.head(10)

In [42]:
#x_hp.info()

In [88]:
x_hp.select_dtypes(include='object').head(10)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
5,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,Shed,WD,Normal
6,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
7,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,...,Attchd,RFn,TA,TA,Y,NaN,NaN,Shed,WD,Normal
8,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,...,Detchd,Unf,Fa,TA,Y,NaN,NaN,NaN,WD,Abnorml
9,RL,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,...,Attchd,RFn,Gd,TA,Y,NaN,NaN,NaN,WD,Normal


In [89]:
x_hp.select_dtypes(include='int64').head(10)

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,706,0,150,856,...,548,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,978,0,284,1262,...,460,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,486,0,434,920,...,608,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,216,0,540,756,...,642,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,655,0,490,1145,...,836,192,84,0,0,0,0,0,12,2008
5,50,14115,5,5,1993,1995,732,0,64,796,...,480,40,30,0,320,0,0,700,10,2009
6,20,10084,8,5,2004,2005,1369,0,317,1686,...,636,255,57,0,0,0,0,0,8,2007
7,60,10382,7,6,1973,1973,859,32,216,1107,...,484,235,204,228,0,0,0,350,11,2009
8,50,6120,7,5,1931,1950,0,0,952,952,...,468,90,0,205,0,0,0,0,4,2008
9,190,7420,5,6,1939,1950,851,0,140,991,...,205,0,4,0,0,0,0,0,1,2008


In [90]:
x_hp.select_dtypes(include='float64').head(10)

,LotFrontage,MasVnrArea,GarageYrBlt
0,65.0,196.0,2003.0
1,80.0,0.0,1976.0
2,68.0,162.0,2001.0
3,60.0,0.0,1998.0
4,84.0,350.0,2000.0
5,85.0,0.0,1993.0
6,75.0,186.0,2004.0
7,NaN,240.0,1973.0
8,51.0,0.0,1931.0
9,50.0,0.0,1939.0


In [91]:
x_hp.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


En la base de datos se cuenta con 36 variables numéricasy 43 categóricas. Luego de leer la descripción de las variables se detecta que MSSubClass es una variable categórica por lo tanto se realiza la conversión. OverallQual con OverallCond son variables ordinales por lo que se decide no cambiarles el tipo.

In [131]:
x_hp["MSSubClass"]=x_hp["MSSubClass"].astype('object')

Se procede ahora a investigar cuántos valores faltanes hay en cada una de ellas para luego decidir cómo proceder. Este paso es importante porque...

In [130]:
obs=x_hp.isnull().sum().sort_values(ascending=False)
percent=(x_hp.isnull().sum()/x_hp.isnull().count()).sort_values(ascending=False)
table=[obs,percent]
pd.concat([obs,percent*100], axis=1,keys=['# Obs.', 'Percent']).head(20)

,# Obs.,Percent
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageCond,81,5.547945
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945


En la tabla enterior se puede observar que 6 variables tienen del 20% a casi el 100% de sus datos nulos. Observando la descripción de las mismas, no parecen ser las más relevanes y en muchos casos gran parte de la información está contenida en otras, entonces se decide excluirlas del análiss. Las otras 12 que tienen datos faltantes en 5% y 0,5% de sus datos, existen diverasas opciones para solucionarlo: 

* Eliminar esas filas en las con na's.
* Completar los na's imputando el valor pormedio o el valor con mayor frecuencia, en estos casos no se tiene en cuenta las correlaciones entre las características y se puede introducir sesgo en los datos. 
* Utilizar k nearest neighbours to predict the values based on how closely it resembles the points in the training set but it is computationally expensive.
* Imputation using Multivariate Imputation by Chained Equation that works by filling the missing data multiple times. Is very flexible and can handle different variables of different data types.
* Imputation Using Deep Learning (Datawig),this method works very well with categorical and non-numerical features. It is a library that learns Machine Learning models using Deep Neural Networks to impute missing values in a dataframe. It also supports both CPU and GPU for training.

First we exclude PoolQC, MiscFeature, Alley, Fence, FireplaceQu and LotFrontage from the data set. Then, we will imputate missing values using Multiple Imputations Chained Equation.

In [146]:
# drop PoolQC, MiscFeature, Alley, Fence, FireplaceQu and LotFrontage
x_hp=x_hp.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage'],axis=1)

In [6]:
import warnings
warnings.filterwarnings('ignore')
# Imputate missing values
from fancyimpute import IterativeImputer


Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\letos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-851573adf4d9>", line 4, in <module>
    from fancyimpute import IterativeImputer
  File "C:\Users\letos\Anaconda3\lib\site-packages\fancyimpute-0.5.4-py3.7.egg\fancyimpute\__init__.py", line 5, in <module>
    from .matrix_factorization import MatrixFactorization
  File "C:\Users\letos\Anaconda3\lib\site-packages\fancyimpute-0.5.4-py3.7.egg\fancyimpute\matrix_factorization.py", line 14, in <module>
    from keras import regularizers
  File "C:\Users\letos\Anaconda3\lib\site-packages\keras\__init__.py", line 3, in <module>
    from . import utils
  File "C:\Users\letos\Anaconda3\lib\site-packages\keras\utils\__init__.py", line 6, in <module>
    from . import conv_utils
  File "C:\Users\letos\Anaconda3\lib\site-packages\keras\utils\conv_utils.py", line 9, in <module>


ModuleNotFoundError: No module named 'absl'

In [5]:
# Initialize IterativeImputer
mice_imputer = IterativeImputer()

x_hp_mice_imputed = x_hp.copy(deep=True)
x_hp_mice_imputed.iloc[:, :] = mice_imputer.fit_transform(x_hp)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\letos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-45772f79fb4a>", line 2, in <module>
    mice_imputer = IterativeImputer()
NameError: name 'IterativeImputer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\letos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\letos\Anaconda3\lib\site-packages\tensorflow-2.1.0-py3.7-win-amd64.egg\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
 

NameError: name 'IterativeImputer' is not defined

In [1]:
from fancyimpute import KNN

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\letos\Anaconda3\lib\site-packages\tensorflow-2.1.0-py3.7-win-amd64.egg\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\letos\Anaconda3\lib\site-packages\tensorflow-2.1.0-py3.7-win-amd64.egg\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\letos\Anaconda3\lib\site-packages\tensorflow-2.1.0-py3.7-win-amd64.egg\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\letos\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\letos\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: No se puede encontrar el mó

TypeError: can only concatenate str (not "list") to str

In [177]:
x_hp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 73 columns):
MSSubClass       1460 non-null object
MSZoning         1460 non-null object
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 non-null object
MasVnrArea       1452 no